# Week 5 - Milestone 1: Smart Tracking System Blockchain Ledger (Submission)

1. Open your Python script and load the CSV file from your Homework 1:

In [24]:
import pandas as pd


# Load IoT sensor data from CSV (Generated in Homework 1)
df = pd.read_csv("smart_tracking_data.csv")

3. Connect Python to the smart contract.
 - Ensure Ganache is running in the background.
 - Establish connection with Web3.py:


In [25]:
from web3 import Web3


# Connect to local blockchain
ganache_url = "http://127.0.0.1:7545"
web3 = Web3(Web3.HTTPProvider(ganache_url))

# Verify connection
if web3.is_connected():
    print("✅ Connected to Ganache successfully!")
else:
    print("❌ Connection failed. Ensure Ganache is running.")

✅ Connected to Ganache successfully!


- Load the smart contract:

In [26]:
# Replace with actual contract address from Remix
contract_address = "0x6127eb02402608ad211d9f2009ed4d6f84d9814d"


# Paste the ABI from Remix
import json

abi_json = '''
[
	{
		"inputs": [],
		"stateMutability": "nonpayable",
		"type": "constructor"
	},
	{
		"anonymous": false,
		"inputs": [
			{
				"indexed": false,
				"internalType": "uint256",
				"name": "timestamp",
				"type": "uint256"
			},
			{
				"indexed": false,
				"internalType": "string",
				"name": "deviceId",
				"type": "string"
			},
			{
				"indexed": false,
				"internalType": "string",
				"name": "dataType",
				"type": "string"
			},
			{
				"indexed": false,
				"internalType": "string",
				"name": "dataValue",
				"type": "string"
			}
		],
		"name": "DataStored",
		"type": "event"
	},
	{
		"inputs": [
			{
				"internalType": "string",
				"name": "_deviceId",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "_dataType",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "_dataValue",
				"type": "string"
			}
		],
		"name": "storeData",
		"outputs": [],
		"stateMutability": "nonpayable",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "",
				"type": "uint256"
			}
		],
		"name": "dataRecords",
		"outputs": [
			{
				"internalType": "uint256",
				"name": "timestamp",
				"type": "uint256"
			},
			{
				"internalType": "string",
				"name": "deviceId",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "dataType",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "dataValue",
				"type": "string"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "index",
				"type": "uint256"
			}
		],
		"name": "getRecord",
		"outputs": [
			{
				"internalType": "uint256",
				"name": "",
				"type": "uint256"
			},
			{
				"internalType": "string",
				"name": "",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "",
				"type": "string"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [],
		"name": "getTotalRecords",
		"outputs": [
			{
				"internalType": "uint256",
				"name": "",
				"type": "uint256"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [],
		"name": "MAX_ENTRIES",
		"outputs": [
			{
				"internalType": "uint256",
				"name": "",
				"type": "uint256"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [],
		"name": "owner",
		"outputs": [
			{
				"internalType": "address",
				"name": "",
				"type": "address"
			}
		],
		"stateMutability": "view",
		"type": "function"
	}
]
'''


# Now load it safely as a Python object
abi = json.loads(abi_json)


# Convert to Python-compatible dict
abi = json.loads(abi_json)


# Convert contract address to checksum format
checksum_address = web3.to_checksum_address(contract_address)

# Load the smart contract
contract = web3.eth.contract(address=checksum_address, abi=abi)


# Set the default sender address (first account from Ganache)
web3.eth.default_account = web3.eth.accounts[0]


print(f"✅ Connected to Smart Contract at {contract_address}")

✅ Connected to Smart Contract at 0x6127eb02402608ad211d9f2009ed4d6f84d9814d


4. Send the IoT data to the blockchain. Each row from the CSV file will be stored as a transaction on the blockchain.
 - Define a function to store IoT data:

In [27]:
import time


def send_iot_data(device_id, data_type, data_value):
    """Sends IoT data to the deployed smart contract"""
    txn = contract.functions.storeData(device_id, data_type, data_value).transact({
        'from': web3.eth.default_account,
        'gas': 3000000
    })
   
    receipt = web3.eth.wait_for_transaction_receipt(txn)
    print(f"✅ Data Stored: {data_type} - {data_value}, Txn Hash: {receipt.transactionHash.hex()}")

 - Loop through the CSV file and send each row to the blockchain:

In [28]:
for _, row in df.iterrows():
    send_iot_data(str(row["device_id"]), "temperature", str(row["temperature"]))
    time.sleep(1)  # Delay to prevent flooding transactions

✅ Data Stored: temperature - 5.12, Txn Hash: 35bc8008d52ac89e4cb758e89f2ed9f7c7ce0273135bbcd2d7b9ccd17e489c31
✅ Data Stored: temperature - 9.51, Txn Hash: b817d120e070d467e4e70614f9128208695dd5127bc189373b42af83ae25c7d8
✅ Data Stored: temperature - 5.87, Txn Hash: 334aa8a7557d8275dbcf3bd9fedff444e4448f7a7ce7f6421a2ca3b90534201d
✅ Data Stored: temperature - 9.5, Txn Hash: 2e586b7032c6977e0da2b7066286fd827eca7a589c39d25d381893fec4fec6bb
✅ Data Stored: temperature - 5.6, Txn Hash: 4fa62f879976dfdfe73c27be48a7e285b198585d51b635f53db7eba898947288
✅ Data Stored: temperature - 7.97, Txn Hash: 6f0e9da8d9e67f9831322b324dd4602a22e559428fcb77e559f49bb1e5b4a04d
✅ Data Stored: temperature - 7.0, Txn Hash: f4ff79559dfa02db4e73e952ddcbbed79b4aeb25055ca9aabdc072a697712af4
✅ Data Stored: temperature - 7.25, Txn Hash: bd5a32388326750777b545f79434b66feee00592080d4c6f601341827e68d4cf
✅ Data Stored: temperature - 4.02, Txn Hash: 7f5edded8174dcede0179c6817753ba7c6c1582c93417d0a399f54233feca9bb
✅ Data Stored

5. Now that the data is on the blockchain, retrieve it to verify storage.
 - Get total stored records:

In [29]:
total_records = contract.functions.getTotalRecords().call()
print(f"Total IoT records stored: {total_records}")

Total IoT records stored: 103


 - Retrieve and print a specific record

In [30]:
record = contract.functions.getRecord(0).call()
print("First Stored Record:", record)

First Stored Record: [1750264279, 'TEST001', 'Temperature', '22.5°C']
